# 导入数据

首先用pandas读取数据

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("data/input/train.csv", encoding='utf8')

In [4]:
df.head()

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低
3,KMT1gFJiU4NWrVDn,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性
4,nVIlGd5yMmc37t1o,17价格忒高，估计也就是14-15左右。,价格,-1,高


In [5]:
df.shape

(9947, 5)

训练集一共有9947行5列，包含id,内容，主题，情感值，情感词

# 分词

使用jieba做中文分词，下面的chinese_word_count用来并行化对每个句子进行分词

In [6]:
import jieba

In [7]:
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

使用apply函数对其中的content列进行中文分词

In [10]:
df["content_cutted"] = df.content.apply(chinese_word_cut)

查看分词结果

In [11]:
df.content_cutted.head()

0    因为 森林 人 即将 换代 ， 这套 系统 没 必要 装在 一款 即将 换代 的 车型 上 ...
1    四驱 价格 貌似 挺 高 的 ， 高 的 可以 看齐 XC60 了 ， 看实车 前 脸 有点...
2    斯柯达 要说 质量 ， 似乎 比 大众 要 好 一点 ， 价格 也 低 一些 ， 用料 完全...
3    这 玩意 都 是 给 有钱 任性 又 不 懂车 的 土豪 用 的 ， 这 价格 换 一次 我...
4               17 价格 忒 高 ， 估计 也 就是 14 - 15 左右 。      
Name: content_cutted, dtype: object

# 文本向量化

使用sklearn中的Tfidf来进行向量化

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [10]:
n_features = 1000

只从文本中提取1000个最为重要的特征关键词

In [15]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)

# LDA

使用LDA指定10个类别，将content划分为10个大类

In [12]:
from sklearn.decomposition import LatentDirichletAllocation

In [13]:
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [14]:
lda.fit(tf)

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


KeyboardInterrupt: 

下面的函数用于将每个主题的前若干个关键词提取出来

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [ ]:
n_top_words = 20

In [ ]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

下面的方法将LDA主题的结果可视化出来

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

In [ ]:
data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.show(data)

# 中文停用词处理

停用词指的是没有什么用的无效单词，使用哈工大的停用词表删除无效单词。下面的函数从文件中获取停用词

In [19]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

In [20]:
stop_words_file = "data/stopwordsHIT.txt"
stopwords = get_custom_stopwords(stop_words_file)

查看最后10个停用词

In [21]:
stopwords[-10:]

['呃', '呗', '咚', '咦', '喏', '啐', '喔唷', '嗬', '嗯', '嗳']

## 查看使用停用词的效果

先初始化向量，对于分词后的词表中的每一个单词作为一列，出现作为1，不出现作为0.term_matrix作为向量化后的数据结构

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

In [23]:
term_matrix = pd.DataFrame(vect.fit_transform(df["content_cutted"]).toarray(), columns=vect.get_feature_names())

In [24]:
term_matrix.head()

,000,01,020,03,030,04,040,05,06,07,...,齐活,齿轮,齿轮油,龌龊,龙华,龙潭,龙膜,龙蟠,龙鼎,龟牌
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
term_matrix.shape

(9947, 15479)

一个有9947个评论，分析后词汇表中有15479个单词

In [26]:
vect = CountVectorizer(stop_words=frozenset(stopwords))

添加停用词后的向量

In [27]:
term_matrix = pd.DataFrame(vect.fit_transform(df["content_cutted"]).toarray(), columns=vect.get_feature_names())

/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [29]:
term_matrix.head()

,000,01,020,03,030,04,040,05,06,07,...,齐活,齿轮,齿轮油,龌龊,龙华,龙潭,龙膜,龙蟠,龙鼎,龟牌
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
term_matrix.shape

(9947, 15274)

减少了200多个单词

## 去掉常见词和稀疏词

去掉文本中出现过于频繁和出现过于稀疏的单词，来减少词表的大小

In [31]:
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 3 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。

In [32]:
vect = CountVectorizer(max_df = max_df,
                       min_df = min_df,
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                       stop_words=frozenset(stopwords))

In [33]:
term_matrix = pd.DataFrame(vect.fit_transform(df["content_cutted"]).toarray(), columns=vect.get_feature_names())

/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [34]:
term_matrix.head

<bound method NDFrame.head of       a180  a4  a4l  a6  a6l  a8  abs  ac  acc  act ...   黑内  黑屏  黑桶  黑色  默认  \
0        0   0    0   0    0   0    0   0    0    0 ...    0   0   0   0   0   
1        0   0    0   0    0   0    0   0    0    0 ...    0   0   0   0   0   
2        0   0    0   0    0   0    0   0    0    0 ...    0   0   0   0   0   
3        0   0    0   0    0   0    0   0    0    0 ...    0   0   0   0   0   
4        0   0    0   0    0   0    0   0    0    0 ...    0   0   0   0   0   
5        0   0    0   0    0   0    0   0    0    0 ...    0   0   0   0   0   
6        0   0    0   0    0   0    0   0    0    0 ...    0   0   0   0   0   
7        0   0    0   0    0   0    0   0    0    0 ...    0   0   0   0   0   
8        0   0    0   0    0   0    0   0    0    0 ...    0   0   0   0   0   
9        0   0    0   0    0   0    0   0    0    0 ...    0   0   0   0   0   
10       0   0    0   0    0   0    0   0    0    0 ...    0   0   0   0   0   
11       0

In [35]:
term_matrix.shape

(9947, 5350)

减少了3分之一的单词

# 训练模型

分类模型使用朴素贝叶斯

In [36]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [37]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(vect, nb)

In [38]:
pipe.steps

[('countvectorizer',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=0.8, max_features=None, min_df=3,
          ngram_range=(1, 1), preprocessor=None,
          stop_words=frozenset({'在下', '哈哈', '［－', '［②⑤］', '尽', '不怕', '着呢', '这时', '慢说', '虽然', '什么', '人家', '［②③］', '『', '嘎', '＜λ', '它', '要是', '［①ｅ］', '】', '⑧', '㈧ ', '兮', '［①ｃ］', '呼哧', '谁', '｝', '］', '■', '［②ｉ］', '倘', '和', '唉', '本着', '［③ａ］', '［①ｆ］', ':', '#', '℃ ', '如', '这就是说', '怎么样', '似的', '经', '总的来说', '*', '【...］', '还有', '进而', '之类', '２．３％', '↑', '加之', '［③ｃ］', '换言之', '趁', '什么样', '待', '了', '罢了', ']', '>', '各种'}),
          strip_accents=None, token_pattern='(?u)\\b[^\\d\\W]\\w+\\b',
          tokenizer=None, vocabulary=None)),
 ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]

In [39]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, df["content_cutted"], df['subject'], cv=5, scoring='accuracy').mean()

/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧',

0.6391920225696252

In [ ]:
test = pd.read_csv("data/test_public.csv",encoding='utf8')

In [ ]:
test.head()

In [ ]:
pipe.fit(df['content_cutted'],test['content'])

In [ ]:
df['content_cutted']((9947,1))

In [ ]:
test['content'].shape